## See the ReadME for a description of the project


### Importing dependencies

In [5]:
import numpy as np
import pandas as pd

### Load Data

In [16]:
train = pd.read_csv('Data/fashion-mnist_train.csv')
test = pd.read_csv('Data/fashion-mnist_test.csv')

train = train.to_numpy()
test = test.to_numpy()


X_train = train[:,1:]
y_train = train[:,0]
X_test = test[:,1:]
y_test = test[:,0]
print("Train data shape:", X_train.shape, "Test data shape:", X_test.shape)
print("Train labels shape:", y_train.shape,"  Test labels shape:", y_test.shape)

Train data shape: (60000, 784) Test data shape: (10000, 784)
Train labels shape: (60000,)   Test labels shape: (10000,)


## Implementing the CNN model

## 